In [48]:
import glob
import os
import re
import numpy as np
import pprint

In [73]:
subgoals = glob.glob("./results/*.subgoal.txt")
for subgoal in subgoals:
    fileid = re.search("./results/sick_trial_(.*).subgoal.txt", subgoal).group(1)
    f = open(subgoal, "r")
    infos=f.readlines()
    f.close()
    subgoalflg = 0
    premises, content_subgoals, relation_subgoals, premise_preds, subgoal_preds, premise_args, subgoal_args= [], [], [], [], [], [], []
    for info in infos:
        if re.search("defined", info):
            premises, content_subgoals, relation_subgoals, premise_preds, subgoal_preds, premise_args, subgoal_args= [], [], [], [], [], [], []
            break
        if re.search("H[0-9]? : ", info) and not "True" in info:
            premise = re.search("H[0-9]? :(.*)\n", info).group(1)
            if premise != "":
                premises.append(premise)
                premise_list = premise.split()
                if len(premise_list) > 0:
                    premise_pred = premise_list[0]
                    premise_preds.append(premise_pred)
                    premise_arg = " ".join(premise_list[1:])
                    premise_args.append(premise_arg)
        if re.search("============================", info) or re.search("subgoal [0-9]* is:", info):
            subgoalflg = 1
            continue
        if subgoalflg == 1:
            if "=" in info:
                #keep 
                relation_subgoals.append(info.strip())
            else:
                subgoal = info.strip()
                if subgoal != "":
                    content_subgoals.append(subgoal)
                subgoal_list = subgoal.split()
                if len(subgoal_list) > 0:
                    subgoal_pred = subgoal_list[0]
                    subgoal_preds.append(subgoal_pred)
                    subgoal_arg = " ".join(subgoal_list[1:])
                    subgoal_args.append(subgoal_arg)
                
    for subgoal_arg in subgoal_args:
        phrase_flg = 0
        if subgoal_arg in premise_args:
            #perfect match
            np_premises = np.array(premises)
            np_subgoals = np.array(content_subgoals)
            match_premise_index = [i for i, x in enumerate(premise_args) if x == subgoal_arg]
            match_subgoal_index = [i for i, x in enumerate(subgoal_args) if x == subgoal_arg]
            print("id: {0}, perfect_pair_candidate, premise: {1}, subgoal: {2}".format(fileid, np_premises[match_premise_index], np_subgoals[match_subgoal_index]))
            phrase_flg = 1
            continue
        else:
            #partial match
            for premise_arg in premise_args:
                if premise_arg in subgoal_arg:
                    match_premise_index = premise_args.index(premise_arg)
                    match_subgoal_index = subgoal_args.index(subgoal_arg)
                    print("id:{0}, partial_pair_candidate, premise: {1}, subgoal: {2}".format(fileid, premises[match_premise_index], content_subgoals[match_subgoal_index]))
                    phrase_flg = 1
                    break
        if phrase_flg == 0:
            match_subgoal_index = subgoal_args.index(subgoal_arg)
            print("id:{0}, no_candidate, premise: {1}, subgoal: {2}".format(fileid, premises, content_subgoals[match_subgoal_index]))

            
    #pprint.pprint(premises)        
    #print("premises:{0}, subgoals:{1}, premise_pred:{2}, subgoal_pred:{3}, premise_arg:{4}, subgoal_arg:{5}"\
    #    .format(premises, content_subgoals, premise_preds, subgoal_preds, premise_args, subgoal_args))

id:1192, no_candidate, premise: [' _walk x0', ' _through x0 x1', ' _wood x1', ' _walk x3', ' _woman (Subj x3)', ' _man (Subj x0)'], subgoal: _area x4
id:1192, partial_pair_candidate, premise:  _walk x0, subgoal: _through x0 x4
id:1192, no_candidate, premise: [' _walk x0', ' _through x0 x1', ' _wood x1', ' _walk x3', ' _woman (Subj x3)', ' _man (Subj x0)'], subgoal: _area x4
id:1236, partial_pair_candidate, premise:  _alone x0, subgoal: _with x0 (Subj x0)
id:1236, partial_pair_candidate, premise:  _alone x0, subgoal: _with x0 (Subj x0)
id: 1266, perfect_pair_candidate, premise: [' _play x0'], subgoal: ['_onstage x0']
id: 1400, perfect_pair_candidate, premise: [' _cry x0'], subgoal: ['_scream x0']
id:1410, no_candidate, premise: [' _cut x1', ' _woman (Subj x1)', ' _shrimp (Acc x1)'], subgoal: _man (Subj ?2791)
id:1410, no_candidate, premise: [' _cut x1', ' _woman (Subj x1)', ' _shrimp (Acc x1)'], subgoal: _butter (Acc ?2791)
id:1410, no_candidate, premise: [' _cut x1', ' _woman (Subj x1)